In [1]:
# Imports
import pandas as pd
import numpy as np 
from datetime import datetime
import matplotlib.pyplot as plt
import io

In [2]:
awaken_tax_config = {
    "platform_name": "Awaken Tax",
    "consolidation_style": "multi", # Multi blockchain consolidation, with multi variables in single column
    "identification_headers": ["Priority", "Provider", "Title", "Hash", "Sent", "Received"],
    "column_mapping": {
        "ID": "Trade_ID_Raw",
        "Priority": "Priority_Raw",
        "Provider": "Exchange_Raw",
        "Title": "Operation_Raw",
        "Date": "DateTime_Raw",
        "Notes": "Comment_Raw",
        "Hash": "Hash_Raw",
        "Cap Gains (USD)": "Cap_Gains_Raw",
        "Sent": "Sent_Raw",
        "Received": "Received_Raw",
        "Fees": "Fee_Raw",
    },
    "target_columns": ['Type', 'Buy', 'Cur.', 'Sell', 'Cur..1', 'Fee', 'Cur..2', 'Exchange', 'Group', 'Comment', 'Date'],
}

In [25]:
def parse_sent_amount(sent):
    """
    Parse the sent amount from the 'Sent' column.
    Returns a tuple of (amount, currency).
    """
    parts = sent.split(", ") if ", " in sent else sent
    if parts != sent and len(parts) >= 2:
        parts2 = parts[0].split(": ") if ": " in parts[0] else parts[0]
    else:
        parts2 = parts.split(": ") if ": " in parts else parts
    if len(parts2) == 2:
        parts3 = parts2[1].split(" ")
        return pd.to_numeric(parts3[0], errors="coerce"), parts3[1]
        
    if pd.isna(sent):
        return None, None
    parts = sent.split()
    if len(parts) == 2:
        return float(parts[0]), parts[1]
    return None, None

In [29]:
print(parse_sent_amount("ðŸ”¥ PhantomWalletSOL_uS: 0.00203928 SOL, ðŸ”¥ PhantomWalletSOL_uS: 24.09263300 USDC (EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v)"))
print(parse_sent_amount("ðŸ§Š LgrWalletSOL_MT: 0.40000000 SOL"))
print(parse_sent_amount("ðŸ§Š LgrWalletSOL_MT: 0.40000000 SOL, ðŸ§Š LgrWalletSOL_MT: 0.33565817 JitoSOL (J1toso1uCk3RLmjorhTtrVwY9HJ7X8V9yYac6Y7kGCPn), ðŸ§Š LgrWalletSOL_MT: 0.00265333 mSOL (mSoLzYCxHdYgdzU16g5QSh3i5K3z3KZK7ytfqcJm7So)"))

(np.float64(0.00203928), 'SOL')
(np.float64(0.4), 'SOL')
(np.float64(0.4), 'SOL')


In [31]:
def parse_received_amount(received):
    """
    Parse the received amount from the 'Received' column.
    The received amount is expected to be in the format 'amount currency'.
    """

    if pd.isna(received):
        return None, None
    #if "wallet" in received.lower():
    parts = received.split(", ") if ", " in received else received
    if parts != received and len(parts) >= 2:
        parts2 = parts[1].split(": ") if ": " in parts[1] else parts[1]
    else:
        parts2 = parts.split(": ") if ": " in parts else parts
    if len(parts2) == 2:
        parts3 = parts2[1].split(" ")
        return pd.to_numeric(parts3[0], errors="coerce"), parts3[1]

    parts = received.split()
    if len(parts) == 2:
        return float(parts[0]), parts[1]
    return None, None

In [32]:
print(parse_received_amount("ðŸ”¥ PhantomWalletSOL_uS: 0.00203928 SOL, ðŸ”¥ PhantomWalletSOL_uS: 24.09263300 USDC (EPjFWdd5AufqSSqeM2qN1xzybapC8G4wEGGkZwyTDt1v)"))
print(parse_received_amount("ðŸ§Š LgrWalletSOL_MT: 0.00203928 SOL, ðŸ§Š LgrWalletSOL_MT: 2997000000.00000000 $CWIF (7atgF8KQo4wJrD5ATGX7t1V2zVvykPJbFfNeVf1icFv1), ðŸ§Š LgrWalletSOL_MT: 0.00308328 SOL"))
print(parse_received_amount("ðŸ§Š LgrWalletSOL_MT: 2997000000.00000000 $CWIF (7atgF8KQo4wJrD5ATGX7t1V2zVvykPJbFfNeVf1icFv1)"))
print(parse_received_amount("Coinbase: 81.00000000 MSOL"))
print(parse_received_amount("Binance US: 1082.92490250 ATOM.S"))

(np.float64(24.092633), 'USDC')
(np.float64(2997000000.0), '$CWIF')
(np.float64(2997000000.0), '$CWIF')
(np.float64(81.0), 'MSOL')
(np.float64(1082.9249025), 'ATOM.S')


In [ ]:
def extract_datetime_combined(dt_str):
    if isinstance(dt_str, (pd.Series, pd.Index)):
        if not dt_str.empty:
            dt_str = dt_str.iloc[0]
        else:
            return ''
    if pd.isna(dt_str) or dt_str == '':
        return ''
    try:
        # Use errors='coerce' to return NaT for unparseable dates
        dt_obj = pd.to_datetime(dt_str, errors='coerce')
        if pd.isna(dt_obj):
            return ''
        return dt_obj.strftime('%d-%m-%Y %H:%M:%S')
    except Exception as e:
        # This block should ideally not be hit with errors='coerce', but good for extreme cases
        print(f"Error formatting datetime '{dt_str}': {e}")
        return ''    
    
def process_multi(input_df, config):
    renamed_df = input_df.rename(columns=config["column_mapping"])
    final_rows = []
    platform = config["platform_name"]
    for _, row in renamed_df.iterrows():
        new_row = {col: '' for col in config["target_columns"]}

        # --- Populate Common Fields ---
        new_row['Date'] = extract_datetime_combined(row.get('DateTime_Raw'))
        #new_row['Trade ID'] = row.get('Trade_ID_Raw', '')
        new_row['Comment'] = row.get('Comment_Raw', '')
        #new_row['Imported From'] = f"{platform} CSV"
        #new_row['Add Date'] = datetime.now().strftime('%Y-%m-%d')
        new_row['Exchange'] = row.get('Exchange_Raw', platform) # Default to platform name, can be overridden
        group = str(row.get('Group_Raw', '')).strip() if not pd.isna(row.get('Group_Raw', '')) else ''
        if (';' in group):
            name, id = group.split(';')
            group = name.strip()
        new_row['Group'] = group

        operation = str(row.get('Operation_Raw', '')).lower()
        category = str(row.get('Category_Raw', '')).lower()
        new_row['Fee'] = pd.to_numeric(row.get('Fee_Raw'), errors='coerce')
        sent_amount, currency = parse_sent_amount(row.get('Sent_Raw', ''))
        received_amount, pair_currency = parse_sent_amount(row.get('Received_Raw', ''))

        if (';' in fee_currency):
            cur, id = fee_currency.split(';')
            fee_currency = cur.strip()
        new_row['Cur..2'] = fee_currency

        if (';' in currency):
            cur, id = currency.split(';')
            currency = cur.strip()
        if (';' in pair_currency):
            cur, id = pair_currency.split(';')
            pair_currency = cur.strip()

        if config["consolidation_style"] == "pair":
            # Like MEXC, when we need to handle pairs separately
            pair = row.get('Pair_Raw', '')
            if pair:
                if '_' in pair:
                    base, quote = pair.split('_')
                elif '-' in pair:
                    base, quote = pair.split('-')
                elif '/' in pair:
                    base, quote = pair.split('/')
                elif ';' in pair:
                    base, quote = pair.split(';')
                else:
                    base, quote = pair.split(' ') if ' ' in pair else (pair, '')
                
                if operation == 'buy':
                    new_row['Cur.'] = base.strip()
                    new_row['Cur..1'] = quote.strip()
                    new_row['Cur..2'] = quote.strip() if fee_currency is None else fee_currency
                elif operation == 'sell':
                    new_row['Cur.'] = quote.strip()
                    new_row['Cur..1'] = base.strip()
                    new_row['Cur..2'] = base.strip() if fee_currency is None else fee_currency
            else:
                new_row['Cur.'] = currency
                new_row['Cur..1'] = pair_currency
        
        if category == 'deposit':
            new_row['Type'] = 'Deposit'
            if (row.get('Primary_Asset_Raw') is not None and row.get('Primary_Amount_Raw') is not None):
                new_row['Buy'] = pd.to_numeric(row.get('Primary_Amount_Raw'), errors='coerce')
                new_row['Cur.'] = row.get('Primary_Asset_Raw')
                print("DEBUG Deposit found")
            else:
                if (currency is not None and (row.get('Buy_Amount_Raw') is not None and row.get('Buy_Amount_Raw') != 0)):
                    new_row['Buy'] = pd.to_numeric(row.get('Buy_Amount_Raw'), errors='coerce')
                    new_row['Cur.'] = currency
                #elif (pair_currency is not None and (row.get('Sell_Amount_Raw') is not None and row.get('Sell_Amount_Raw') != 0)):
                    #new_row['Buy'] = pd.to_numeric(row.get('Sell_Amount_Raw'), errors='coerce')
                    #new_row['Cur.'] = pair_currency
        elif category == 'withdrawal':
            new_row['Type'] = 'Withdrawal'
            if (row.get('Primary_Asset_Raw') is not None and row.get('Primary_Amount_Raw') is not None):
                new_row['Sell'] = pd.to_numeric(row.get('Primary_Amount_Raw'), errors='coerce')
                new_row['Cur..1'] = row.get('Primary_Asset_Raw')
            else:
                if (pair_currency is not None and (row.get('Sell_Amount_Raw') is not None and row.get('Sell_Amount_Raw') != 0)):
                    new_row['Sell'] = pd.to_numeric(row.get('Sell_Amount_Raw'), errors='coerce')
                    new_row['Cur..1'] = pair_currency
                #elif (currency is not None and (row.get('Buy_Amount_Raw') is not None and row.get('Buy_Amount_Raw') != 0)):
                    #new_row['Sell'] = pd.to_numeric(row.get('Buy_Amount_Raw'), errors='coerce')
                    #new_row['Cur..1'] = currency
        elif category == 'convert':
            if (currency.lower() == "w" + pair_currency.lower()) or (pair_currency.lower() == "w" + currency.lower()):
                new_row['Type'] = 'Swap (non taxable)'  # For conversions, we treat them as swaps
                # For Binance 'Convert', Base is what you sold, Quote is what you bought
                new_row['Sell'] = pd.to_numeric(row.get('Buy_Amount_Raw'), errors='coerce')
                new_row['Cur..1'] = currency
                new_row['Buy'] = pd.to_numeric(row.get('Sell_Amount_Raw'), errors='coerce')
                new_row['Cur.'] = pair_currency
            else:
                new_row['Type'] = 'Trade'
                new_row['Sell'] = pd.to_numeric(row.get('Buy_Amount_Raw'), errors='coerce')
                new_row['Buy'] = pd.to_numeric(row.get('Sell_Amount_Raw'), errors='coerce')
                if not config["consolidation_style"] == "pair":
                    new_row['Cur..1'] = currency
                    new_row['Cur.'] = pair_currency
        else: # trade
            new_row['Type'] = 'Trade'
            if operation is None or operation == '' or operation == 'buy':
                # If operation is not specified, assume it's normal buy trade
                new_row['Buy'] = pd.to_numeric(row.get('Buy_Amount_Raw'), errors='coerce')
                new_row['Sell'] = pd.to_numeric(row.get('Sell_Amount_Raw'), errors='coerce')
                if not config["consolidation_style"] == "pair":
                    new_row['Cur.'] = currency
                    new_row['Cur..1'] = pair_currency
            elif operation == 'sell':
                new_row['Sell'] = pd.to_numeric(row.get('Buy_Amount_Raw'), errors='coerce')
                new_row['Buy'] = pd.to_numeric(row.get('Sell_Amount_Raw'), errors='coerce')
                if not config["consolidation_style"] == "pair":
                    new_row['Cur..1'] = currency
                    new_row['Cur.'] = pair_currency
      
        final_rows.append(new_row)

    final_df = pd.DataFrame(final_rows, columns=config["target_columns"])

    # Final cleaning and sorting
    for col in ['Buy', 'Sell', 'Fee']:
        if col in final_df.columns:
            final_df[col] = pd.to_numeric(final_df[col], errors='coerce').fillna(0)

    if not final_df.empty and 'Date' in final_df.columns:
        final_df['Sort_DateTime'] = pd.to_datetime(final_df['Date'], format='%d-%m-%Y %H:%M:%S', errors='coerce')
        final_df = final_df.sort_values(by='Sort_DateTime', na_position='first').drop(columns=['Sort_DateTime'])
        final_df['Date'] = final_df['Date'].astype(str)

    return final_df

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'  # Update the path as needed
#from pytesseract import image_to_string

from PIL import Image, ImageFilter, ImageEnhance

# Load image and extract text
image = Image.open('image4.png')
extracted_text = pytesseract.image_to_string(image)
print(extracted_text)
#image = image.filter(ImageFilter.MedianFilter())
#enhancer = ImageEnhance.Contrast(image)
#image = enhancer.enhance(2.0)  # Increase contrast
#image = image.convert('1')
#image.save('temp4.png')
#extracted_text = pytesseract.image_to_string(Image.open('temp4.png'))

Amount

0.000275

756

68.05

17585975

0.000256

0.03212

0.070028



In [4]:
import pytesseract
from PIL import Image, ImageFilter, ImageEnhance

# Set the path to your Tesseract installation
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the original image
image = Image.open('coinbase4.png')

# --- 1. Pre-processing Steps ---

# Convert to grayscale
gray_image = image.convert('L')

# Upscale the image (e.g., 2x). This helps recognize small characters.
width, height = gray_image.size
upscaled_image = gray_image.resize((width * 2, height * 2), Image.Resampling.LANCZOS)

# Binarize the image (convert to pure black and white)
# The threshold value (e.g., 150) may need tweaking for best results.
threshold = 130
bw_image = upscaled_image.point(lambda x: 0 if x < threshold else 255, '1')

# --- 2. OCR Extraction with Configuration ---

# Specify Page Segmentation Mode 6 for uniform text blocks
custom_config = r'--psm 6'

extracted_text = pytesseract.image_to_string(bw_image, config=custom_config)

print("--- Extracted Text after Pre-processing ---")
print(extracted_text)
print(extracted_text.replace(' ', ', '))

--- Extracted Text after Pre-processing ---
Balance Vv
$2,699.52
9,801.13150484 HBAR
$1,874.60
0.01972463 BTC
$1,731.24
1,995.09222 ADA
$1,605.09
4,603.079136 ALGO
$1,304.07
616.000416 XRP
$1,231.14
56,971,141.89300279 SHIB

Balance, Vv
$2,699.52
9,801.13150484, HBAR
$1,874.60
0.01972463, BTC
$1,731.24
1,995.09222, ADA
$1,605.09
4,603.079136, ALGO
$1,304.07
616.000416, XRP
$1,231.14
56,971,141.89300279, SHIB



In [8]:
import pytesseract
import cv2
import numpy as np
from PIL import Image

# Set the path to your Tesseract installation
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Load the original image
image = Image.open('image4.png')

# Convert PIL Image to an OpenCV image format
# We still use grayscale as the base
gray_image = image.convert('L')
cv_image = np.array(gray_image)

# Apply adaptive thresholding
# This will create a clean black-and-white image that adapts to local brightness
processed_image = cv2.adaptiveThreshold(
    cv_image, 255,
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY,
    11, # The neighborhood size (block size)
    2   # A constant subtracted from the mean
)

# Configuration for Tesseract
custom_config = r'--psm 6'

# Run OCR on the processed image
extracted_text = pytesseract.image_to_string(processed_image, config=custom_config)

print("--- Extracted Text after Adaptive Thresholding ---")
print(extracted_text)

--- Extracted Text after Adaptive Thresholding ---
Pamount]
£756)



In [4]:
import cv2
import pytesseract

pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Grayscale, Gaussian blur, Otsu's threshold
image = cv2.imread('image4.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Morph open to remove noise and invert image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
invert = 255 - opening

# Perform text extraction
data = pytesseract.image_to_string(invert, lang='eng', config='--psm 6')
print(data)

cv2.imshow('thresh', thresh)
cv2.imshow('opening', opening)
cv2.imshow('invert', invert)
#cv2.waitKey()

smount.
oon
7
an
DA
‘CoonTsS
oom
coors

